In [ ]:
# !pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error 

In [ ]:
train_df = pd.read_feather('data_feather/train_df_3.feather')
train_target_df = pd.read_feather('data_feather/train_target.feather')
train_df.head()

In [ ]:
train_df = pd.read_feather('data_feather/train_df_4.feather')
train_target_df = pd.read_feather('data_feather/train_target.feather')
train_df.head()

In [ ]:
train_target_df.head()

In [ ]:
train_full = train_df.merge(train_target_df, on='postcode')
print(train_full.shape)
train_full.head()

In [ ]:
X_train_full = train_full.values[:,3:-1]
y_train_full = train_full.values[:,-1]


In [ ]:
np.save( 'data/X_train_full_1.npy',X_train_full,)
np.save( 'data/y_train_full_1.npy',y_train_full,)

In [ ]:
X_train_full = np.load('data/X_train_full_1.npy', allow_pickle=True)
y_train_full = np.load('data/X_train_full_1.npy', allow_pickle=True)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split( train_full.values[:,3:-1], train_full.values[:,-1], test_size=0.2, random_state=35)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
X_val1, X_val2, y_val1, y_val2 = train_test_split( X_val, y_val, test_size=0.5, random_state=35)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_val1.shape, X_val2.shape, y_val1.shape, y_val2.shape

In [ ]:
np.save( 'data/X_train_1.npy',X_train,)
np.save( 'data/y_train_1.npy',y_train,)
np.save('data/X_val.npy',X_val,)
np.save('data/y_val.npy',y_val,)
np.save( 'data/X_val1_1.npy',X_val1,)
np.save( 'data/y_val1_1.npy',y_val1,)
np.save( 'data/X_val2_1.npy',X_val2,)
np.save('data/y_val2_1.npy',y_val2, )


In [ ]:
X_train = np.load( 'data/X_train_1.npy',allow_pickle=True)
y_train = np.load( 'data/y_train_1.npy',allow_pickle=True)
X_val = np.load('data/X_val.npy',allow_pickle=True)
y_val = np.load('data/y_val.npy',allow_pickle=True)
X_val1 = np.load( 'data/X_val1_1.npy',allow_pickle=True)
y_val1 = np.load( 'data/y_val1_1.npy',allow_pickle=True)
X_val2 = np.load( 'data/X_val2_1.npy',allow_pickle=True)
y_val2 = np.load('data/y_val2_1.npy', allow_pickle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_val1.shape, X_val2.shape, y_val1.shape, y_val2.shape

In [ ]:
X_train = np.load( 'data/X_train_2.npy',allow_pickle=True)
y_train = np.load( 'data/y_train_2.npy',allow_pickle=True)
X_val = np.load('data/X_val_2.npy',allow_pickle=True)
y_val = np.load('data/y_val_2.npy',allow_pickle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
d_train = xgb.DMatrix(X_train, label=y_train)
d_val = xgb.DMatrix(X_val, label=y_val)
d_val1 = xgb.DMatrix(X_val1, label=y_val1)
d_val2 = xgb.DMatrix(X_val2, label=y_val2)
watchlist = [(d_train, 'train'), (d_val, 'eval')]
# params = {'max_depth': 3, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}
# xgb_model = xgb.train(num_boost_round=1000, dtrain=d_train, evals=watchlist, early_stopping_rounds=50, maximize=False, params=params)

In [ ]:
#'max_depth': 6, 'objective': 'reg:squaredlogerror', 'eval_metric': 'rmse'
# 0.46558806626158705
# 0.49094714691258773
# 0.471836257996428
# {'max_depth': 3, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}
# 0.47554992352440545
# 0.48907461822641735
# 0.47194289469467293
# 0.4746581844849312
# 0.48902032413111546
# 0.4718937041219335
train_predict = xgb_model.predict(d_train, ntree_limit=xgb_model.best_ntree_limit)
val1_predict = xgb_model.predict(d_val1, ntree_limit=xgb_model.best_ntree_limit)
val2_predict = xgb_model.predict(d_val2, ntree_limit=xgb_model.best_ntree_limit)
print(np.sqrt(mean_squared_error(y_train, train_predict)))
print(np.sqrt(mean_squared_error(y_val1, val1_predict)))
print(np.sqrt(mean_squared_error(y_val2, val2_predict)))

### Optuna

In [ ]:
!pip install optuna

1. xgb-study_ver1 - wrongly added some columns twice
2. xgb-study_ver2 - fixed it 

In [ ]:
import optuna
import logging
import sys
import os

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

study_name = "xgb-study_ver2"

storage_name = "sqlite:///{}/{}.db".format(os.getcwd(), study_name) 





In [ ]:
optuna.delete_study(study_name, storage_name)

In [ ]:
study = optuna.load_study(study_name="xgb-study_ver1",
storage="sqlite:///{}/{}.db".format(os.getcwd(), "xgb-study_ver1") ,)

In [ ]:

study = optuna.create_study(
study_name=study_name,
storage=storage_name,
# load_if_exists=True,
direction="minimize",
)

In [ ]:

def objective(trial):

    param = {
    "verbosity": 0, # "n_jobs": 7,
    "objective": "reg:squarederror",
    #defines booster, gblinear for linear functions.
    # "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear","dart"]), 
    # "booster": trial.suggest_categorical("booster", ["gbtree","dart"]),#
    "booster":"gbtree",
    # L2 regularization weight.
    # "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log-True),
    "lambda": trial.suggest_float("lambda",1e-5, 1, log= True), # L1 regularization weight. # "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log-True),
    "alpha": trial.suggest_float("alpha", 1e-5, 1, log=True), 
    #sampling ratio for training data.
    # "subsample": trial.suggest_float("subsample", 8.2, 1.0),
    "subsample":trial.suggest_float("subsample", 0.2, 1.0), # sampling according to each tree. # "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    'eval_metric': 'rmse',
    }

    if param["booster"] in ["gbtree", "dart"]: # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 2, 9,)
        # param["max_depth"] = trial.suggest_categorical("max_depth", [2,3]) #minimum child weight, larger the term more conservative the tree.
        #param["min_child_weight"] trial.suggest_int("min_child_weight", 2, 10) 
        param["min_child_weight"] = trial.suggest_int("min_child_weight",10, 20)
        #param["eta"] trial.suggest_float("eta", le-8, 1.0, log-True)
        param["eta"] = trial.suggest_float("eta", 1e-5, 1.0)
        #defines how selective algorithm is.
        param["gamma"] = trial.suggest_loguniform("gamma", 1e-5, 10) 
        # param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"]== "dart":

        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"]) 
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"]) 
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"]= trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    xgb_model = xgb.train(
        param,
        dtrain=d_train,
        evals = watchlist,
        early_stopping_rounds=50,
        num_boost_round = 1000, maximize = False, verbose_eval=False)
    
    if param["booster"] in  ["gbtree", "dart"]:
        trial.set_user_attr('best_iteration', xgb_model.best_iteration)
        train_predict = xgb_model.predict(d_train, iteration_range = (0, xgb_model.best_iteration))
        val_predict = xgb_model.predict(d_val, iteration_range = (0,xgb_model.best_iteration))
        # val2_predict = xgb_model.predict(d_val2, ntree_limit=xgb_model.best_ntree_limit)
    else:
        train_predict = xgb_model.predict(d_train)
        val_predict = xgb_model.predict(d_val)

    train_rmse = np.sqrt(mean_squared_error(y_train, train_predict))
    val_rmse = np.sqrt(mean_squared_error(y_val, val_predict))
    # val2_rmse = np.sqrt(mean_squared_error(y_val2, val2_predict)))

    trial.set_user_attr('train_rmse',train_rmse)
    trial.set_user_attr('val_rmse',val_rmse)

    return val_rmse


In [ ]:
study.optimize(objective, n_trials = 20, show_progress_bar=True)

trials_df = study.trials_dataframe()

trials_df.to_excel(study_name + '.xlsx') 

In [ ]:
trials_df = study.trials_dataframe()
trials_df['user_attrs_val_rmse'].min()

In [ ]:
trials_df.tail(25)

In [ ]:
trials_df.head()

In [ ]:
trials_df['params_max_depth'].value_counts()

In [ ]:
print(study.best_params)
print(study.best_trial.user_attrs)

In [ ]:
print(study.best_params)
print(study.best_trial.user_attrs)

{'alpha': 0.0005602249720412078, 'colsample_bytree': 0.4808460431302214, 'eta': 0.08922767256703615, 'gamma': 0.00018863733126355857, 'lambda': 49.477279807044034, 'max_depth': 3, 'min_child_weight': 587, 'subsample': 0.9466709857360812}
{'best_iteration': 130, 'train_rmse': 0.47487325235529615, 'val_rmse': 0.47987409985548724}

In [ ]:
trials_df.shape

In [ ]:

# params = study.best_params
watchlist = [(d_train, 'train'), (d_val, 'eval')]
params = {'alpha': 0.035688937871948614, 'colsample_bytree': 0.5578828917109813, 'eta': 0.1883793073253146, 'gamma': 0.4049115972788908, 'lambda': 0.5022115278674546, 'max_depth': 4, 'min_child_weight': 122, 'subsample': 0.8953059917041677}
xgb_model_selected = xgb.train(num_boost_round = 100, dtrain=d_train, evals=watchlist, maximize=False, params=params, early_stopping_rounds=50)
# study.best_trial.user_attrs['best_iteration']

In [ ]:
xgb_model_selected.best_iteration

In [ ]:
## Load file from tuned version from kaggle
import joblib
# xgb_model_selected = joblib.load("data_feather/xgb_model_selected_ver3.pkl")
xgb_model_selected = joblib.load("data_feather/xgb_model_selected_ver4.pkl")

In [ ]:
# idv_columns = ['All usual residents','Males','Females','Lives in a household','Lives in a communal establishment','Schoolchild or full-time student aged 4 and over at their non term-time address','Area (Hectares)','Density (number of persons per hectare)','roadClassi_','roadClassi_A Road','roadClassi_Motorway','roadFuncti_','roadFuncti_A Road','roadFuncti_Motorway','formOfWay_','formOfWay_Collapsed Dual Carriageway','formOfWay_Dual Carriageway','formOfWay_Roundabout','formOfWay_Single Carriageway','formOfWay_Slip Road','length_min','length_mean','length_max','distance to the nearest point on rd_min','distance to the nearest point on rd_mean','distance to the nearest point on rd_max','primaryRou_or','primaryRou_and','Day_of_Week_1','Day_of_Week_2','Day_of_Week_3','Day_of_Week_4','Day_of_Week_5','Day_of_Week_6','Day_of_Week_7','1st_Road_Class_1','1st_Road_Class_3','1st_Road_Class_4','1st_Road_Class_5','1st_Road_Class_6','2nd_Road_Class_-1','2nd_Road_Class_3','2nd_Road_Class_4','2nd_Road_Class_5','2nd_Road_Class_6','Pedestrian_Crossing-Human_Control_Control by other authorised person','Pedestrian_Crossing-Human_Control_Control by school crossing patrol','Pedestrian_Crossing-Human_Control_None within 50 metres','Pedestrian_Crossing-Physical_Facilities_Central refuge','Pedestrian_Crossing-Physical_Facilities_Footbridge or subway','Pedestrian_Crossing-Physical_Facilities_No physical crossing within 50 meters','Pedestrian_Crossing-Physical_Facilities_Pedestrian phase at traffic signal junction','Pedestrian_Crossing-Physical_Facilities_Zebra crossing','Pedestrian_Crossing-Physical_Facilities_non-junction pedestrian crossing','Light_Conditions_Darkeness: No street lighting','Light_Conditions_Darkness: Street lighting unknown','Light_Conditions_Darkness: Street lights present and lit','Light_Conditions_Darkness: Street lights present but unlit','Light_Conditions_Daylight: Street light present','Weather_Conditions_Fine with high winds','Weather_Conditions_Fine without high winds','Weather_Conditions_Fog or mist','Weather_Conditions_Other','Weather_Conditions_Raining with high winds','Weather_Conditions_Raining without high winds','Weather_Conditions_Snowing with high winds','Weather_Conditions_Snowing without high winds','Weather_Conditions_Unknown','Road_Surface_Conditions_','Road_Surface_Conditions_Dry','Road_Surface_Conditions_Flood (Over 3cm of water)','Road_Surface_Conditions_Frost/Ice','Road_Surface_Conditions_Snow','Road_Surface_Conditions_Wet/Damp','Special_Conditions_at_Site_','Special_Conditions_at_Site_Auto traffic signal partly defective','Special_Conditions_at_Site_Auto traffic singal out','Special_Conditions_at_Site_Mud','Special_Conditions_at_Site_None','Special_Conditions_at_Site_Ol or diesel','Special_Conditions_at_Site_Permanent sign or marking defective or obscured','Special_Conditions_at_Site_Road surface defective','Special_Conditions_at_Site_Roadworks','Carriageway_Hazards_Any animal (except a ridden horse)','Carriageway_Hazards_Dislodged vehicle load in carriageway','Carriageway_Hazards_Involvement with previous accident','Carriageway_Hazards_None','Carriageway_Hazards_Other object in carriageway','Carriageway_Hazards_Pedestrian in carriageway (not injured)','state_Alba / Scotland','state_Cymru / Wales','state_England','Road_Type_Dual carriageway','Road_Type_One way street','Road_Type_Roundabout','Road_Type_Single carriageway','Road_Type_Slip road','Road_Type_Unknown','Speed_limit_20','Speed_limit_30','Speed_limit_40','Speed_limit_50','Speed_limit_60','Speed_limit_70','Urban_or_Rural_Area_1','Urban_or_Rural_Area_2','Police_Force_min','Police_Force_mean','Police_Force_max','Number_of_Vehicles_min','Number_of_Vehicles_mean','Number_of_Vehicles_max','Police_Force_sum','Number_of_Vehicles_sum','Did_Police_Officer_Attend_Scene_of_Accident_or','Did_Police_Officer_Attend_Scene_of_Accident_and','Road_Type_Dual carriageway_count','Road_Type_One way street_count','Road_Type_Roundabout_count','Road_Type_Single carriageway_count','Road_Type_Slip road_count','Road_Type_Unknown_count','Speed_limit_20_count','Speed_limit_30_count','Speed_limit_40_count','Speed_limit_50_count','Speed_limit_60_count','Speed_limit_70_count','Urban_or_Rural_Area_1_count','Urban_or_Rural_Area_2_count','Police_Force_sum/Number_of_Vehicles_sum','Police_Force_mean/Number_of_Vehicles_mean','Time_category_0_8_count','Time_category_16_24_count','Time_category_8_16_count','1st_Road_Number_count>100']
idv_columns = ['All usual residents','Males','Females','Lives in a household','Lives in a communal establishment','Schoolchild or full-time student aged 4 and over at their non term-time address','Area (Hectares)','Density (number of persons per hectare)','roadClassi_','roadClassi_A Road','roadClassi_Motorway','roadFuncti_','roadFuncti_A Road','roadFuncti_Motorway','formOfWay_','formOfWay_Collapsed Dual Carriageway','formOfWay_Dual Carriageway','formOfWay_Roundabout','formOfWay_Single Carriageway','formOfWay_Slip Road','length_min','length_mean','length_max','distance to the nearest point on rd_min','distance to the nearest point on rd_mean','distance to the nearest point on rd_max','primaryRou_or','primaryRou_and','Day_of_Week_1','Day_of_Week_2','Day_of_Week_3','Day_of_Week_4','Day_of_Week_5','Day_of_Week_6','Day_of_Week_7','1st_Road_Class_1','1st_Road_Class_3','1st_Road_Class_4','1st_Road_Class_5','1st_Road_Class_6','2nd_Road_Class_-1','2nd_Road_Class_3','2nd_Road_Class_4','2nd_Road_Class_5','2nd_Road_Class_6','Pedestrian_Crossing-Human_Control_Control by other authorised person','Pedestrian_Crossing-Human_Control_Control by school crossing patrol','Pedestrian_Crossing-Human_Control_None within 50 metres','Pedestrian_Crossing-Physical_Facilities_Central refuge','Pedestrian_Crossing-Physical_Facilities_Footbridge or subway','Pedestrian_Crossing-Physical_Facilities_No physical crossing within 50 meters','Pedestrian_Crossing-Physical_Facilities_Pedestrian phase at traffic signal junction','Pedestrian_Crossing-Physical_Facilities_Zebra crossing','Pedestrian_Crossing-Physical_Facilities_non-junction pedestrian crossing','Light_Conditions_Darkeness: No street lighting','Light_Conditions_Darkness: Street lighting unknown','Light_Conditions_Darkness: Street lights present and lit','Light_Conditions_Darkness: Street lights present but unlit','Light_Conditions_Daylight: Street light present','Weather_Conditions_Fine with high winds','Weather_Conditions_Fine without high winds','Weather_Conditions_Fog or mist','Weather_Conditions_Other','Weather_Conditions_Raining with high winds','Weather_Conditions_Raining without high winds','Weather_Conditions_Snowing with high winds','Weather_Conditions_Snowing without high winds','Weather_Conditions_Unknown','Road_Surface_Conditions_','Road_Surface_Conditions_Dry','Road_Surface_Conditions_Flood (Over 3cm of water)','Road_Surface_Conditions_Frost/Ice','Road_Surface_Conditions_Snow','Road_Surface_Conditions_Wet/Damp','Special_Conditions_at_Site_','Special_Conditions_at_Site_Auto traffic signal partly defective','Special_Conditions_at_Site_Auto traffic singal out','Special_Conditions_at_Site_Mud','Special_Conditions_at_Site_None','Special_Conditions_at_Site_Ol or diesel','Special_Conditions_at_Site_Permanent sign or marking defective or obscured','Special_Conditions_at_Site_Road surface defective','Special_Conditions_at_Site_Roadworks','Carriageway_Hazards_Any animal (except a ridden horse)','Carriageway_Hazards_Dislodged vehicle load in carriageway','Carriageway_Hazards_Involvement with previous accident','Carriageway_Hazards_None','Carriageway_Hazards_Other object in carriageway','Carriageway_Hazards_Pedestrian in carriageway (not injured)','state_Alba / Scotland','state_Cymru / Wales','state_England','Road_Type_Dual carriageway','Road_Type_One way street','Road_Type_Roundabout','Road_Type_Single carriageway','Road_Type_Slip road','Road_Type_Unknown','Speed_limit_20','Speed_limit_30','Speed_limit_40','Speed_limit_50','Speed_limit_60','Speed_limit_70','Urban_or_Rural_Area_1','Urban_or_Rural_Area_2','Police_Force_min','Police_Force_mean','Police_Force_max','Number_of_Vehicles_min','Number_of_Vehicles_mean','Number_of_Vehicles_max','Police_Force_sum','Number_of_Vehicles_sum','Did_Police_Officer_Attend_Scene_of_Accident_or','Did_Police_Officer_Attend_Scene_of_Accident_and','Road_Type_Dual carriageway_count','Road_Type_One way street_count','Road_Type_Roundabout_count','Road_Type_Single carriageway_count','Road_Type_Slip road_count','Road_Type_Unknown_count','Speed_limit_20_count','Speed_limit_30_count','Speed_limit_40_count','Speed_limit_50_count','Speed_limit_60_count','Speed_limit_70_count','Urban_or_Rural_Area_1_count','Urban_or_Rural_Area_2_count','Police_Force_sum/Number_of_Vehicles_sum','Police_Force_mean/Number_of_Vehicles_mean','Time_category_0_8_count','Time_category_16_24_count','Time_category_8_16_count','1st_Road_Number_count>100','1st_Road_Class_t_enc','Urban_or_Rural_Area_t_enc','Speed_limit_t_enc','Day_of_Week_t_enc','Time_category_t_enc','No. Of Accidents','Time_category_0_8','Time_category_16_24','Time_category_8_16','Day_of_Week_1_and_Time_category_0_8','Day_of_Week_1_and_Time_category_16_24','Day_of_Week_1_and_Time_category_8_16','Day_of_Week_2_and_Time_category_0_8','Day_of_Week_2_and_Time_category_16_24','Day_of_Week_2_and_Time_category_8_16','Day_of_Week_3_and_Time_category_0_8','Day_of_Week_3_and_Time_category_16_24','Day_of_Week_3_and_Time_category_8_16','Day_of_Week_4_and_Time_category_0_8','Day_of_Week_4_and_Time_category_16_24','Day_of_Week_4_and_Time_category_8_16','Day_of_Week_5_and_Time_category_0_8','Day_of_Week_5_and_Time_category_16_24','Day_of_Week_5_and_Time_category_8_16','Day_of_Week_6_and_Time_category_0_8','Day_of_Week_6_and_Time_category_16_24','Day_of_Week_6_and_Time_category_8_16','Day_of_Week_7_and_Time_category_0_8','Day_of_Week_7_and_Time_category_16_24','Day_of_Week_7_and_Time_category_8_16',]
d_train = xgb.DMatrix(X_train, label=y_train, feature_names = idv_columns)
d_val = xgb.DMatrix(X_val, label=y_val, feature_names = idv_columns)
# d_val1 = xgb.DMatrix(X_val1, label=y_val1, feature_names = idv_columns)
# d_val2 = xgb.DMatrix(X_val2, label=y_val2, feature_names = idv_columns)
# watchlist = [(d_train, 'train'), (d_val, 'eval')]

In [ ]:
xgb_model_selected.num_boosted_rounds, xgb_model_selected.best_ntree_limit, xgb_model_selected.best_iteration

In [ ]:
train_predict = xgb_model_selected.predict(d_train, iteration_range = (0, xgb_model_selected.best_iteration + 1)) #somehow plus 1 is giving better results
val_predict = xgb_model_selected.predict(d_val,iteration_range = (0, xgb_model_selected.best_iteration + 1))

train_rmse = np.sqrt(mean_squared_error(y_train, train_predict))
val_rmse = np.sqrt(mean_squared_error(y_val, val_predict))

print(train_rmse, val_rmse)

In [ ]:
d = xgb_model_selected.get_score(importance_type='gain')
print(type(d))
# d

In [ ]:
feature_imp_df = pd.DataFrame(d, index=[0])
feature_imp_df =  feature_imp_df.transpose()


feature_imp_df = feature_imp_df.sort_values(ascending=False, by=0)

feature_imp_df.head(20)

In [ ]:
np.save('data_feather/Val_predict_xgb_ver2.npy',val_predict, )

In [ ]:
test_df = pd.read_feather('data_feather/test_df_processed_combined_2.feather')

In [ ]:
test_df = pd.read_feather('data_feather/test_df_processed_2.feather')

In [ ]:
test_df.shape

In [ ]:
for col in test_df.columns:
    if 'Pedestrian_Crossing-Human_Control' in col:
        print(col)

In [ ]:
test_df = test_df.drop(['Pedestrian_Crossing-Human_Control_Control by other authorised person_y','Pedestrian_Crossing-Human_Control_Control by school crossing patrol_y','Pedestrian_Crossing-Human_Control_None within 50 metres_y',], axis = 1)


test_df = test_df.rename({'Pedestrian_Crossing-Human_Control_Control by other authorised person_x':'Pedestrian_Crossing-Human_Control_Control by other authorised person','Pedestrian_Crossing-Human_Control_Control by school crossing patrol_x':'Pedestrian_Crossing-Human_Control_Control by school crossing patrol','Pedestrian_Crossing-Human_Control_None within 50 metres_x':'Pedestrian_Crossing-Human_Control_None within 50 metres',}, axis = 1)
test_df.shape

In [ ]:
test_df.to_feather('data_feather/test_df_processed.feather')

In [ ]:
test_df.head()

In [ ]:
X_train.shape

In [ ]:
X_test = test_df.values[:,3:]
d_test = xgb.DMatrix(X_test, feature_names=idv_columns)
test_predict = xgb_model_selected.predict(d_test, iteration_range = (0, xgb_model_selected.best_iteration + 1))

test_predict[:10]

In [ ]:
test_predict.shape

In [ ]:
test_predict_df = pd.DataFrame(test_df.iloc[:,0])
test_predict_df.head()

In [ ]:
test_predict_df['Accident_risk_index'] = test_predict

In [ ]:
test_predict_df.head()

In [ ]:
# test_predict_df = pd.read_csv('data_out/my_submission_9.csv')
# test_predict_df.head()

In [ ]:
test_predict_df.to_csv('data_out/my_submission_9.csv', index=False)

In [ ]:
test_predict_df.to_csv('data_out/my_submission_10.csv', index=False)

### Full train fit

In [ ]:
d_train_full = xgb.DMatrix(X_train_full, label=y_train_full)

In [ ]:
params = study.best_params
xgb_model_selected = xgb.train(num_boost_round=159+1, dtrain=d_train_full, maximize=False, params=params, verbose_eval=False)

In [ ]:
train_predict = xgb_model_selected.predict(d_train, iteration_range = (0, xgb_model_selected.best_iteration + 1))
val_predict = xgb_model_selected.predict(d_val,iteration_range = (0, xgb_model_selected.best_iteration + 1))

train_rmse = np.sqrt(mean_squared_error(y_train, train_predict))
val_rmse = np.sqrt(mean_squared_error(y_val, val_predict))

train_rmse, val_rmse

In [ ]:
test_predict = xgb_model_selected.predict(d_test, ntree_limit=xgb_model_selected.best_iteration+1)
test_predict_df = pd.DataFrame(test_df.iloc[:,0])
test_predict_df['Accident_risk_index'] = test_predict
# test_predict_df.head()

test_predict_df.to_csv('data_out/my_submission_7.csv', index=False)

In [ ]:
d = xgb_model_selected.get_score(importance_type='gain')
print(type(d))
d

In [ ]:
feature_imp_df = pd.DataFrame(d, index=[0])
feature_imp_df =  feature_imp_df.transpose()
feature_imp_df

In [ ]:
feature_imp_df.sort_values(ascending=False, by=0)

In [ ]:
train_df.iloc[:,[111,39,106]]

In [ ]:

(test_df.columns == train_df.columns).all()